# Call, annotate, and analyze peaks for each cell type

To analyze the scATAC-seq data, we'll need to call peaks and identify TF motifs in each peak. To facilitate downstream analyses, a single set of peaks will be called for all cells obtained for each cell type. This will give us a single set of features to use for differential testing for each type.

Once we have peaks, we'll analyze them from 3 angles: 
1. Identify differentially accessible sites induced by drug treatment at each time point.
2. Calculate enrichment of motif annotations in differentially-acessible peaks.
3. Compute Peak-to-gene correlations using the paired scRNA-seq data from our TEA-seq experiments.

Along the way, we'll save each of the stages of analysis for downstream work and visualization:
- Peaks and annotations
- Differential peaks and annotation enrichments
- Peak-to-gene correlations

# Setup

To call peaks, we'll need to install MACS2

In [ ]:
system("pip install --upgrade --force-reinstall MACS2")

## Load packages

hise: The Human Immune System Explorer R SDK package  
dplyr: Dataframe handling functions   
ArchR: scATAC-seq analysis  
purrr: Functional programming tools  


In [ ]:
quiet_library <- function(...) { suppressPackageStartupMessages(library(...)) }
quiet_library(hise)
quiet_library(dplyr)
quiet_library(ArchR)
quiet_library(BSgenome.Hsapiens.UCSC.hg38)
quiet_library(purrr)

## Retrieve files

Now, we'll use the HISE SDK package to retrieve the TEA-seq ArchR Projects based on their file UUIDs. These will be placed in the `cache/` subdirectory by default.

In [ ]:
atac_file_uuids <- list(
    "11235534-d09d-4c57-a648-20fb13317eab",
    "2d1a00ca-f1f6-41c1-9691-f37916fad00c",
    "365045be-e8a6-4a4d-9fe1-b31b7593799a",
    "403e1064-34ea-4992-8752-6d1ddb9fb614",
    "49d66578-bc16-4840-871c-25de96456f83",
    "f1a32e62-d2e9-4052-b971-dd960d605d70"
)

In [ ]:
fres <- hise::cacheFiles(
    atac_file_uuids
)

In [ ]:
atac_tar_files <- map(fres, "filePath")

In [ ]:
walk(
    atac_tar_files,
    function(tf) {
        command <- paste("tar -xf", tf)
        system(command)
    }
)

Note: un-tar-ing the files moves them to the `output` directory, based on their original filenames.

In [ ]:
type_paths <- list.files(
    "output",
    full.names = TRUE
)

In [ ]:
cell_types <- sub(".+-(.+)_20.+", "\\1", type_paths)
cell_types

In [ ]:
type_proj <- map(
    type_paths,
    loadArchRProject,
    showLogo = FALSE
)
names(type_proj) <- cell_types

# Run analysis per cell type

## Peak calling and annotation

In [ ]:
addArchRVerbose(verbose = FALSE)
addArchRThreads(14)
addArchRGenome("hg38")

In [ ]:
type_proj <- map(
    type_proj,
    function(proj) {
        message("Adding coverage")
        proj <- addGroupCoverages(proj, groupBy = "Sample", force = TRUE)
        message("Adding peak set")
        proj <- addReproduciblePeakSet(proj, groupBy = "Sample", force = TRUE)
        message("Adding peak matrix")
        proj <- addPeakMatrix(proj, force = TRUE)
        message("Adding Peak Annotations")
        proj <- addMotifAnnotations(
            proj, 
            motifSet = "cisbp", 
            name = "Motif",
            force = TRUE
        )
        proj <- saveArchRProject(proj)
    }
)

Extract and output peaks and annotations matrix for downstream use

In [ ]:
type_peak_gr <- map(
    type_proj,
    getPeakSet
)

In [ ]:
dir.create("output")

In [ ]:
peak_output_files <- paste0(
    "output/peak-GRanges-",
    cell_types,
    "_", Sys.Date(),
    ".rds")

In [ ]:
walk2(
    type_peak_gr,
    peak_output_files,
    saveRDS
)

In [ ]:
type_motif_mat <- map(
    type_proj,
    function(proj) {
        anno <- getPeakAnnotation(proj)
        matches <- readRDS(anno$Matches)
        assays(matches)$matches
    }
)

In [ ]:
motif_mat_output_files <- paste0(
    "output/peak-motif-matches-",
    cell_types,
    "_", Sys.Date(),
    ".rds")

In [ ]:
walk2(
    type_motif_mat,
    motif_mat_output_files,
    saveRDS
)

## Differential peak accessibility

To test for DAPs, we'll need to include the treatment and timepoint metadata for each sample in the ArchR projects:

In [ ]:
sample_manifest <- read.csv("../common/EXP00454_TEAseq_sample_manifest.csv")

In [ ]:
sample_meta <- sample_manifest %>%
  select(pbmc_sample_id, treatment, timepoint) %>%
  mutate(Sample = paste0("EXP-00454-P1_", pbmc_sample_id),
         treat_time = paste0(treatment, "_", timepoint)) %>%
  select(-pbmc_sample_id)
head(sample_meta)

In [ ]:
type_proj <- map(
    type_proj,
    function(proj) {
        proj_meta <- as.data.frame(getCellColData(proj))
        cell_names <- rownames(proj_meta)

        proj_meta <- proj_meta %>%
          select(Sample) %>%
          left_join(sample_meta)

        addCellColData(proj, proj_meta$treat_time, "treat_time", cell_names, force = TRUE)
        
    }
)

Next, we can define the foreground and background conditions for the DAP tests:

In [ ]:
fg_treat_times <- c("bortezomib_4", "lenalidomide_4", "dexamethasone_4",
                    "bortezomib_24", "lenalidomide_24", "dexamethasone_24",
                    "bortezomib_72", "lenalidomide_72")
bg_treat_times <- c(rep("dmso_4", 3),
                    rep("dmso_24", 3),
                    rep("dmso_72", 2))

And we'll define this helper function for running the tests and retrieving the results.

Note that the cutoff of FDR < 2 is there to retrieve results for all peaks regardless of FDR, since all FDR values will be 1 or lower.

In [ ]:
run_dap_test <- function(fg, bg, proj) {

    message(paste(fg, "vs", bg))
    
    suppressMessages(
        getMarkerFeatures(
            proj,
            useMatrix = "PeakMatrix",
            groupBy = "treat_time",
            useGroups = fg,
            bgdGroups = bg
        )
    )
    
}

In [ ]:
table(getCellColData(type_proj[[1]])$treat_time)

In [ ]:
all_dap_results <- map(
    type_proj,
    function(proj) {
        ct <- getCellColData(proj)$aifi_cell_type[1]
        message(ct)
        
        dap_results <- map2(
            fg_treat_times,
            bg_treat_times,
            run_dap_test,
            proj = proj
        )
        
        dap_results

    }
)

Extract DAP results to data.frame

In [ ]:
all_dap_df <- map2_dfr(
    all_dap_results,
    cell_types,
    function(dap, ct) {
        dap_df <- pmap_dfr(
            list(fg = fg_treat_times,
                 bg = bg_treat_times,
                 res = dap),
            function(fg, bg, res) {
                group_marker_res <- getMarkers(
                    res, 
                    cutOff = "FDR < 2")
                
                group_marker_df <- as.data.frame(group_marker_res[[1]])
                group_marker_df$fg <- fg
                group_marker_df$bg <- bg

                group_marker_df
            }
        )

        dap_df$aifi_cell_type <- ct
        dap_df
    }
)

In [ ]:
all_dap_df <- all_dap_df %>%
  dplyr::rename(logFC = Log2FC,
                adjP = FDR) %>%
  select(aifi_cell_type, fg, bg, seqnames, start, end, logFC, adjP, MeanDiff, idx)

In [ ]:
dap_output_file <- paste0(
    "output/all-archr-dap_",
    Sys.Date(),
    ".csv")

In [ ]:
write.csv(all_dap_df,
          dap_output_file,
          quote = FALSE,
          row.names = FALSE)

## Motif annotation enrichment

For each set of differentially accessible peaks, we'll calculate differentially enriched motifs.

To account for the differences in cell count that affect sensitivity of DAP results, we'll use the top 500 peaks from each comparison to identify DEMs.

We'll also separate peaks with increased accessibility from peaks with decreased accessibility in the DAP results.

The helper function, below, does the bulk of this work:

In [ ]:
top_dap_directional_dem <- function(fg, bg, dap, proj, top_n = 500) {

    message(paste(fg, "vs", bg))

    group_marker_res <- getMarkers(
        dap, 
        cutOff = "FDR < 2")
    
    peak_df <- as.data.frame(group_marker_res[[1]])
    names(peak_df) <- names(assays(dap))
    
    up_peaks <- peak_df %>%
      filter(Log2FC > 0) %>%
      arrange(Pval) %>%
      head(top_n)
    up_cut <- up_peaks$Pval[top_n]
    
    up_enriched_motifs <- suppressMessages(
        peakAnnoEnrichment(
            seMarker = dap,
            ArchRProj = proj,
            peakAnnotation = "Motif",
            cutOff = paste("Pval < ", up_cut,"& Log2FC > 0")
    ))
    
    up_res <- as.data.frame(as.list(assays(up_enriched_motifs)))
    names(up_res) <- names(assays(up_enriched_motifs))
    up_res$direction <- "up"
    
    dn_peaks <- peak_df %>%
      filter(Log2FC < 0) %>%
      arrange(Pval) %>%
      head(top_n)
    dn_cut <- dn_peaks$Pval[top_n]
    
    dn_enriched_motifs <- suppressMessages(
        peakAnnoEnrichment(
            seMarker = dap,
            ArchRProj = proj,
            peakAnnotation = "Motif",
            cutOff = paste("Pval < ", dn_cut, "& Log2FC < 0")
    ))
    
    dn_res <- as.data.frame(as.list(assays(dn_enriched_motifs)))
    names(dn_res) <- names(assays(dn_enriched_motifs))
    dn_res$direction <- "dn"
    
    res <- rbind(up_res, dn_res)
    res$fg <- fg
    res$bg <- bg

    res
}

Here, we'll iterate over each cell type (the first `map2_dfr` call), and each comparison (the `pmap_dfr` call):

In [ ]:
all_dem_res <- map2_dfr(
    type_proj,
    all_dap_results,
    function(proj, type_dap) {
        ct <- getCellColData(proj)$aifi_cell_type[1]
        message(ct)
         
        dem_res <- pmap_dfr(
            list(fg = fg_treat_times,
                 bg = bg_treat_times,
                 dap = type_dap),
            top_dap_directional_dem,
            proj = proj,
            top_n = 500
        )

        dem_res$aifi_cell_type <- ct
        dem_res
    }
)

In [ ]:
all_dem_res <- all_dem_res %>%
  mutate(
      nomP = 10^(-mlog10p),
      adjP = 10^(-mlog10Padj),
      tf_gene = sub("_.+", "", feature)
  ) %>%
  select(aifi_cell_type, fg, bg, direction,
         feature, tf_gene, Enrichment, nomP, adjP,
         everything())

In [ ]:
head(all_dem_res)

In [ ]:
dem_output_file <- paste0(
    "output/all-archr-dem_",
    Sys.Date(),
    ".csv")

In [ ]:
write.csv(all_dem_res,
          dem_output_file,
          quote = FALSE,
          row.names = FALSE)

## Peak-to-Gene correlation

Finally, we'll test peak-to-gene correlation by integrating the scRNA-seq data from TEA-seq. This will be matched at the single-cell level to the cells in the ArchR Projects. First, we'll need to retrieve the scRNA-seq matrices from HISE:

In [ ]:
scrna_file_ids <- list(
    "7bdac6ef-e5e5-4150-b4f3-9c1a1e250334", # CD4 data
    "46438bc4-cde6-4ae6-b349-9c513dd9d16f" # CD8 data
)

In [ ]:
scrna_file_res <- hise::cacheFiles(
    scrna_file_ids
)

In [ ]:
scrna_files <- map(scrna_file_res, "filePath")

In [ ]:
so_list <- map(scrna_files, readRDS)

In [ ]:
count_mat <- cbind(
    so_list[[1]][["RNA"]]@counts,
    so_list[[2]][["RNA"]]@counts
)

Before integration, we'll need to filter for genes that are in the gene annotations for the ArchR Projects.

We'll then be able to use the counts and the gene GRanges to build the SummarizedExperiment objects that ArchR requires.

In [ ]:
gene_gr <- type_proj[[1]]@geneAnnotation$genes
gene_gr <- gene_gr[gene_gr$symbol %in% rownames(count_mat)]
count_mat <- count_mat[gene_gr$symbol,]

In [ ]:
str(count_mat)

For each cell type, we'll select the cells for that type using barcodes, then match the cell names to those used by ArchR so that the data are compatible.

In [ ]:
type_proj <- map(
    type_proj,
    function(proj) {
        proj_meta <- as.data.frame(getCellColData(proj))
        rna_mat <- count_mat[,proj_meta$barcodes]
        colnames(rna_mat) <- rownames(proj_meta)

        serna <- SummarizedExperiment(
            assays = SimpleList(counts = rna_mat),
            rowRanges = gene_gr
        )
        
        addGeneExpressionMatrix(
            proj,
            serna
        )
    })

Now that we have gene expression in the object, we can compute peak-to-gene correlations. This step requires a reduced dimensionality projection to find neighborhoods of cells, so we'll also compute LSI for each cell type.

In [ ]:
type_proj <- map(
    type_proj,
    addIterativeLSI
)

In [ ]:
type_proj <- map(
    type_proj,
    addPeak2GeneLinks,
    useMatrix = "GeneExpressionMatrix"
)

We'll use this helper function to extract the Peak2Gene results using an absolute correlation cutoff. This is more permissive than the methods built in to ArchR, as it will allow us to retrieve negative correlations as well as positive correlations. These can be difficult to interpret, but let's keep them at this point.

It's also nice to have the position of the peak, gene symbol, and distance from peak to gene for thinking about the results. This function will also collate that information from the peak and gene GRanges objects.

In [ ]:
get_p2g_links <- function(proj, abs_cutoff = 0.1) {
    p2g <- metadata(proj@peakSet)$Peak2GeneLinks
    peaks <- metadata(p2g)$peakSet
    genes <- metadata(p2g)$geneSet
    p2g_df <- as.data.frame(p2g)
    p2g_df <- p2g_df %>%
      filter(abs(Correlation) > 0.1)
    p2g_df <- p2g_df %>%
      mutate(gene = as.character(genes$name[idxRNA]),
             seqnames = as.character(seqnames(peaks)[idxATAC]),
             start = start(peaks)[idxATAC],
             end = end(peaks)[idxATAC]) %>%
      mutate(
         distance = pmin(
             abs(start(peaks)[idxATAC] - start(genes)[idxRNA]),
             abs(end(peaks)[idxATAC] - start(genes)[idxRNA])
         )
      )
}

In [ ]:
type_p2g <- map2(
    type_proj,
    cell_types,
    function(proj, ct) {
        p2g_df <- get_p2g_links(
            proj,
            abs_cutoff = 0.1
        )
        p2g_df$aifi_cell_type <- ct
        p2g_df
    }
)

In [ ]:
p2g_output_files <- paste0(
    "output/peak-to-gene-",
    cell_types,
    "_", Sys.Date(),
    ".csv")

In [ ]:
walk2(
    type_p2g,
    p2g_output_files,
    write.csv,
    quote = FALSE,
    row.names = FALSE
)

### Store results in HISE

Update folder names for ArchR Projects and bundle the files as .tar for upload.

In [ ]:
out_paths <- sub("20.+",Sys.Date(),type_paths)

In [ ]:
walk2(
    type_paths,
    out_paths,
    function(type_path, out_path) {
        file.rename(type_path, out_path)
        out_tar <- paste0(out_path, ".tar")

        command <- paste(
            "tar -cf",
            out_tar,
            out_path
        )

        system(command)
    }
)

In [ ]:
study_space_uuid <- "40df6403-29f0-4b45-ab7d-f46d420c422e"
title <- paste("VRd TEA-seq T Cell ATAC Analysis", Sys.Date())

In [ ]:
out_list <- as.list(
    list.files(
        "output", 
        pattern = ".csv$|.rds$|.tar$",
        full.names = TRUE
))
input_ids <- c(atac_file_uuids, scrna_file_ids)

In [ ]:
uploadFiles(
    files = out_list,
    studySpaceId = study_space_uuid,
    title = title,
    inputFileIds = input_ids,
    store = "project",
    doPrompt = FALSE
)

# Session Info

In [ ]:
sessionInfo()